In [1]:
from konlpy.tag import Okt

In [4]:
okt = Okt()
token = okt.morphs("나는 자연어 처리를 배운다")
print(token)

['나', '는', '자연어', '처리', '를', '배운다']


In [5]:
word2index = {}
for voca in token:
    if voca not in word2index.keys():
        word2index[voca] = len(word2index)
print(word2index)

{'나': 0, '는': 1, '자연어': 2, '처리': 3, '를': 4, '배운다': 5}


In [6]:
def one_hot_encoding(word, word2index):
    one_hot_vector = [0] * (len(word2index))
    index = word2index[word]
    one_hot_vector[index] = 1
    return one_hot_vector

In [7]:
one_hot_encoding("자연어", word2index)

[0, 0, 1, 0, 0, 0]

### 희소표현
- 표현하고자 하는 단어의 인덱스만 1이고 나머지는 0인 벡터

In [8]:
import torch

In [9]:
dog = torch.FloatTensor([1, 0, 0, 0, 0])
cat = torch.FloatTensor([0, 1, 0, 0, 0])
computer = torch.FloatTensor([0, 0, 1, 0, 0])
netbook = torch.FloatTensor([0, 0, 0, 1, 0])
book = torch.FloatTensor([0, 0, 0, 0, 1])

In [10]:
# cosine_similarity : 코사인 유사성을 반환()
print(torch.cosine_similarity(dog, cat, dim=0))
print(torch.cosine_similarity(cat, computer, dim=0))
print(torch.cosine_similarity(computer, netbook, dim=0))
print(torch.cosine_similarity(netbook, book, dim=0))

tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)


### 밀집표현
- 사용자가 설정한 값으로 모든 단어의 벡터 표현의 차원을 맞춤
- 실수값을 가지게 됨

### 워드 임베딩
- 단어를 밀집 벡터의 형태로 표현하는 방법

# 워드 투 벡터(Word2Vec)
- 단어 간 유사도를 반영
### 분산 표현
- 비슷한 위치에서 등장하는 단어들은 비슷한 의미를 가진다.

## 종류

### CBOW(Continuous Bag of Words)
   - 주변에 있는 단어들을 가지고, 중간에 있는 단어들을 예측하는 방법
   
### Skip-Gram
   - 중간에 있는 단어로 주변 단어들을 예측하는 방법

In [11]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [12]:
import urllib.request
import zipfile
from lxml import etree # xml파일을 다루기 위한 라이브러리
import re
from nltk.tokenize import word_tokenize, sent_tokenize

In [13]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/GaoleMeng/RNN-and-FFNN-textClassification/master/ted_en-20160408.xml", filename="ted_en-20160408.xml")

('ted_en-20160408.xml', <http.client.HTTPMessage at 0x1a43cb2da60>)

In [16]:
targetXML = open('ted_en-20160408.xml', 'r', encoding='UTF8')

target_text = etree.parse(targetXML)
# xml 파일로부터 <content>와 </content> 사이의 내용만 가져온다.
parse_text = '\n'.join(target_text.xpath('//content/text()'))

# 괄호제거 정규표현식 : r'\([^)]*\)'
# re.sub(정규표현식, 치환할 문자, 치환될 대상)
content_text = re.sub(r'\([^)]*\)', '', parse_text)

sent_text = sent_tokenize(content_text)

normalized_text = []
for string in sent_text:
    tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
    normalized_text.append(tokens)
    
result = []
result = [word_tokenize(sentence) for sentence in normalized_text]

In [17]:
print('총 샘플의 갯수 : {}'.format(len(result)))

총 샘플의 갯수 : 273424


In [18]:
for line in result[:3]:
    print(line)

['here', 'are', 'two', 'reasons', 'companies', 'fail', 'they', 'only', 'do', 'more', 'of', 'the', 'same', 'or', 'they', 'only', 'do', 'what', 's', 'new']
['to', 'me', 'the', 'real', 'real', 'solution', 'to', 'quality', 'growth', 'is', 'figuring', 'out', 'the', 'balance', 'between', 'two', 'activities', 'exploration', 'and', 'exploitation']
['both', 'are', 'necessary', 'but', 'it', 'can', 'be', 'too', 'much', 'of', 'a', 'good', 'thing']


### gensim : Word2Vec를 지원하는 패키지
- 손쉽게 단어를 임베딩 벡터로 변환가능

Word2Vec 
- size = 워드 벡터의 특징 값. 즉, 임베딩 된 벡터의 차원.
- window = 컨텍스트 윈도우 크기
- min_count = 단어 최소 빈도 수 제한 (빈도가 적은 단어들은 학습하지 않는다.)
- workers = 학습을 위한 프로세스 수
- sg = 0은 CBOW, 1은 Skip-gram.

In [21]:
from gensim.models import Word2Vec
model = Word2Vec(sentences=result, size = 100, window=5, min_count=5, workers=4, sg=0)

In [23]:
# model.wv.most_similar : 가장 유사한 단어 출력
model_result = model.wv.most_similar("man")
print(model_result)

[('woman', 0.8489661812782288), ('guy', 0.8201931715011597), ('boy', 0.7743191719055176), ('lady', 0.7660802602767944), ('girl', 0.7516182661056519), ('soldier', 0.7428873777389526), ('gentleman', 0.7136614322662354), ('kid', 0.6987431049346924), ('poet', 0.6751818656921387), ('philosopher', 0.651231050491333)]


### Word2Vec 모델 저장하고 로드하는 방법

In [25]:
from gensim.models import KeyedVectors
model.wv.save_word2vec_format('./eng_w2v') # 모델 저장
loaded_model = KeyedVectors.load_word2vec_format("eng_w2v") # 모델 로드

In [27]:
model_result = loaded_model.most_similar("man")
print(model_result)

[('woman', 0.8489661812782288), ('guy', 0.8201931715011597), ('boy', 0.7743191719055176), ('lady', 0.7660802602767944), ('girl', 0.7516182661056519), ('soldier', 0.7428873777389526), ('gentleman', 0.7136614322662354), ('kid', 0.6987431049346924), ('poet', 0.6751818656921387), ('philosopher', 0.651231050491333)]


In [32]:
f = open(r'C:\Users\admin\wiki_data.txt', encoding="utf8")

In [34]:
i = 0
while True:
    line = f.readline()
    if line != '\n':
        i = i + 1
        print("%d번째 줄 : "%i + line)
    if i == 5:
        break
f.close()

1번째 줄 : <doc id="5" url="https://ko.wikipedia.org/wiki?curid=5" title="지미 카터">

2번째 줄 : 지미 카터

3번째 줄 : 제임스 얼 "지미" 카터 주니어(, 1924년 10월 1일 ~ )는 민주당 출신 미국 39번째 대통령 (1977년 ~ 1981년)이다.

4번째 줄 : 지미 카터는 조지아주 섬터 카운티 플레인스 마을에서 태어났다. 조지아 공과대학교를 졸업하였다. 그 후 해군에 들어가 전함·원자력·잠수함의 승무원으로 일하였다. 1953년 미국 해군 대위로 예편하였고 이후 땅콩·면화 등을 가꿔 많은 돈을 벌었다. 그의 별명이 "땅콩 농부" (Peanut Farmer)로 알려졌다.

5번째 줄 : 1962년 조지아 주 상원 의원 선거에서 낙선하나 그 선거가 부정선거 였음을 입증하게 되어 당선되고, 1966년 조지아 주 지사 선거에 낙선하지만 1970년 조지아 주 지사를 역임했다. 대통령이 되기 전 조지아주 상원의원을 두번 연임했으며, 1971년부터 1975년까지 조지아 지사로 근무했다. 조지아 주지사로 지내면서, 미국에 사는 흑인 등용법을 내세웠다.



1) morphs : 형태소 추출\
2) pos : 품사 태깅(Part-of-speech tagging)\
3) nouns : 명사 추출

In [36]:
from konlpy.tag import Okt
okt = Okt()

fread = open(r'C:\Users\admin\wiki_data.txt', encoding="utf8")
n = 0
result = []

while True:
    line = fread.readline()
    if n == 100000:
        break
    n = n + 1
    if n % 5000 == 0:
        print("%d번째 while문." %n)
    #  norm은 문장을 정규화(이미지에서 0~1.0사이값 나오도록). stem은 각 단어에서 어간을 추출
    tokenlist = okt.pos(line, stem=True, norm=True)
    temp = []
    for word in tokenlist:
        if word[1] in ["Noun"]:
            temp.append((word[0]))
            
    if temp:
        result.append(temp)
fread.close()

5000번째 while문.
10000번째 while문.
15000번째 while문.
20000번째 while문.
25000번째 while문.
30000번째 while문.
35000번째 while문.
40000번째 while문.
45000번째 while문.
50000번째 while문.
55000번째 while문.
60000번째 while문.
65000번째 while문.
70000번째 while문.
75000번째 while문.
80000번째 while문.
85000번째 while문.
90000번째 while문.
95000번째 while문.
100000번째 while문.


In [37]:
print('총 샘플의 개수 : {}'.format(len(result)))

총 샘플의 개수 : 40283


In [38]:
from gensim.models import Word2Vec
model = Word2Vec(result, size=100, window=5, min_count=5, workers=4, sg=0)

In [39]:
model_result1 = model.wv.most_similar("대한민국")
print(model_result1)

[('조선민주주의인민공화국', 0.8123728632926941), ('북한', 0.8041773438453674), ('수교', 0.7786298990249634), ('중화인민공화국', 0.7784789800643921), ('중화민국', 0.7668930292129517), ('독도', 0.762294352054596), ('자국', 0.76041179895401), ('개국', 0.7549357414245605), ('유엔', 0.7438442707061768), ('국기', 0.7422870397567749)]
